In [180]:
import math
import openpyxl
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sp
from scipy.stats import binom, expon, weibull_min, lognorm, norm
from tabulate import tabulate
import pandas as pd

In [181]:
def Update():
    # Model_List must be imported every time
    data = openpyxl.load_workbook("Model_List_Updated_.xlsx")
    read_data = data.active

    Probability = []
    Fail_Time = []

    for row in range(1, read_data.max_row):
        for col in read_data.iter_cols(2, 2):
            Probability.append(col[row].value)

    for row in range(1, read_data.max_row):
        for col in read_data.iter_cols(3, 3):
            Fail_Time.append(col[row].value)

    Probability = [
        0.99931506849315, 0.999868149, 0.999868149, 0.999868149, 0.999868149, 1, 1, 1, 1,
        0.999996546, 0.999996546, 1, 1, 1, 1, 0.999994046, 0.999994046, 0.9999, 0.999996546,
        1, 1, 0.999982209, 0.9999, 0.9999, 0.9999, 0.999982209, 0.999982209, 0.9999, 0.999982209,
        0.999982209, 0.999247479, 0.999897429, 0.999247479, 0.999897429, 0.999247479, 0.999897429,
        1, 0.999312485, 0.999312485, 0.999312485, 0.999312485, 0.999312485, 0.999312485, 0.999897429,
        0.999897429, 0.999989539, 0.999897429, 0.999897429, 0.999875595, 0.999875595
    ]
    

    return [Probability, Fail_Time]

In [182]:
def percentage_of_year_to_time(percentage):
    if not 0 <= percentage <= 100:
        raise ValueError("Percentage must be between 0 and 100.")

    seconds_in_a_year = 365 * 24 * 60 * 60
    total_seconds = (percentage / 100) * seconds_in_a_year

    days, remainder = divmod(total_seconds, 24 * 60 * 60)
    hours, remainder = divmod(remainder, 60 * 60)
    minutes, seconds = divmod(remainder, 60)

    return int(days), int(hours), int(minutes), int(seconds)

In [183]:
def test_weibull(prob_fail):
    prob = weibull_min.rvs(prob_fail, 1)
    return 1 if prob_fail == 1 or prob >= 1.01 else 0

def test_binom(prob_fail):
    return binom.rvs(1, prob_fail)

def test_expon(prob_fail):
    return 1 if prob_fail == 1 else 0 if expon.rvs(1 / prob_fail) < 1.01 else 1

def run_lognormal(MTTR):
    if mean_lognormal == 0:
        sigma = 1
    else: 
        sigma = 0.5*mean_lognormal
    mu = np.log(mean_lognormal) - (sigma**2) / 2
    time = lognorm(s=sigma, scale=np.exp(mu)).rvs()
    return time

In [184]:
## Tests the Electrical side of the model. This is checked before HVAC.
def Failure_Elec(PoF, Carry):
    # 1,2,3,4 are the generators, only 2 are needed
    if PoF == 1 or PoF == 2 or PoF == 3 or PoF == 4:
        # Checks if there are 3 gens that fail
        if (Carry[1] + Carry[2] + Carry[3] + Carry[4]) == 3:
            Carry[PoF] += 1
            return [0, Carry]
        # If less than 2 fail, increases the carry
        Carry[PoF] += 1
        return [1, Carry]
    # Checks the busses, only one needed, same process as gens
    elif PoF == 5 or PoF == 6:
        if (Carry[5] or Carry[6]) == 1:
            Carry[PoF] += 1
            return [0, Carry]
        Carry[PoF] += 1
        return [1, Carry]
    # Checks UPS, only one needed
    elif PoF == 7 or PoF == 8:
        if (Carry[7] or Carry[8]) == 1:
            Carry[PoF] += 1
            return [0, Carry]
        Carry[PoF] += 1
        return [1, Carry]
    # If none of the components are those listed above, there is no exception
    # therefore, they fail
    else:
        return [0, Carry]

In [185]:
# Checks HVAC side, same as electrical
def Failure_HVAC(PoF, Carry):
    # Check for Watertowers
    if PoF == 30 or PoF == 32 or PoF == 34:
        Carry[PoF] += 1
        return [1, Carry]
    # Checks Pumps
    elif PoF == 31 or PoF == 33 or PoF == 35:
        Carry[PoF] += 1
        # Checks if those pumps that are connected to water towers are down together
        if (Carry[30] == 0 and Carry[31] == 0) or (Carry[32] == 0 and Carry[33] == 0) or (Carry[34] == 0 and Carry[35] == 0):
            return [1, Carry]
        return [0, Carry]
    # Checks Direct Chillers
    elif PoF == 40 or PoF == 37 or PoF == 38 or PoF == 39:
        if (Carry[40] + Carry[37] + Carry[38] + Carry[39]) == 4:
            Carry[PoF] += 1
            return [0, Carry]
        Carry[PoF] += 1
        return [1, Carry]
    # Checks Backup Chillers
    elif PoF == 41 or PoF == 42:
        if (Carry[41] or Carry[42]) == 1:
            Carry[PoF] += 1
            return [0, Carry]
        Carry[PoF] += 1
        return [1, Carry]
    # Checks CRAH's
    elif PoF == 48 or PoF == 49:
        if (Carry[48] or Carry[49]) == 1:
            Carry[PoF] += 1
            return [0, Carry]
        Carry[PoF] += 1
        return [1, Carry]
    else:
        return [0, Carry]

In [186]:
def Simulation(modelNum):
    # Allows for the import of the list of PoF and Failtime
    pull = Update()
    # set the numbers of fails to 0
    numFails = 0
    #print(pull[1])
    Prob = pull[0]
    Fail_Time = pull[1]
    # Sets the hour
    hour = 0
    # Sets the list for each hour
    Year_Output = []
    # Creates an empty list with the amount of components to count failures
    Fail_List = [0] * len(Prob)
    while hour < (24 * 365):
        i = 0
        test_mult = 1
        elec_mult = 0
        Comp_Test = []
        # Tests every component and adds them to the Comp_Test list
        while i < len(Prob):
            if modelNum == 1:
                temp = test_expon(Prob[i])
            if modelNum == 0.75:
                temp = test_expon(Prob[i]*0.75)
            if modelNum == 1.25:
                temp = test_expon(Prob[i]*1.25)
            if modelNum == 2:
                temp = test_weibull(Prob[i])
            if modelNum == 1.75:
                temp = test_weibull(Prob[i]*0.75)
            if modelNum == 2.25:
                temp = test_weibull(Prob[i]*1.25)
            ##if modelNum is 3
            else:
                temp = test_binom(Prob[i])
            Comp_Test.append(temp)
            test_mult *= temp
            # Does the comp for only electrical
            if i == 29:
                elec_mult = test_mult
            i = i + 1
        # If every component passes, passes the hour
        if test_mult == 1:
            Year_Output.extend([1])
            hour += 1
        else:
            # Sees if positions need to be skipped
            if Comp_Test[0] == 1:
                # If all electrical is good, start with HVAC
                if elec_mult == 1:
                    i = 29
                # Else, start after gens since com power is good
                else:
                    i = 5
            # If Com Power is down, start with Com Power
            else:
                i = 0
            Fail_Carry = [0] * len(Prob)
            # Tests Electrical slide, last electrical comp is 29
            while i < 30:
                # Does comp Fail
                if Comp_Test[i] == 0:
                    temp = Fail_List[i]
                    Fail_List[i] = temp + 1
                    con = Failure_Elec(i, Fail_Carry)
                    # Gets info from Failure_Elec and sees if failed
                    if con[0] == 0 and i != 0:
                        Year_Output.extend([0] * Fail_Time[i])
                        failure_duration = float(run_lognormal(Fail_Time[i]))
                        #print(str(Fail_Time[i]) + "-> " + str(failure_duration))
                        hour += failure_duration
                        numFails += 1
                        i = 30
                    # Keeps the carry and continues
                    else:
                        Fail_Carry = con[1]
                i += 1
                # Catches loop if all comps work, starting with 30
                if i == 30:
                    while i < len(Prob):
                        if Comp_Test[i] == 0:
                            temp = Fail_List[i]
                            Fail_List[i] = temp + 1
                            con = Failure_HVAC(i, Fail_Carry)
                            if con[0] == 0:
                                Year_Output.extend([0] * Fail_Time[i])
                                failure_duration = float(run_lognormal(Fail_Time[i]))
                                #print(str(Fail_Time[i]) + "-> " + str(failure_duration))
                                hour += failure_duration
                                numFails += 1
                                i = 49
                            else:
                                Fail_Carry = con[1]
                        i += 1
    # Returns the Total amount of hours passed in a year and the list of fails
    #print("Number of Fails per year:" + str(numFails))
    return [sum(Year_Output), Fail_List, numFails]

def run_multi(years, modelNum):
    i = 0
    Output = 0
    List = []
    while i < years:
        temp = Simulation(modelNum)
        Output = Output + temp[0]
        List.append(temp[1])
        i+=1
    Fail_List = [0]*50
    for row in range(years):
        for col in range(len(Fail_List)):
            fl = Fail_List[col]
            Fail_List[col] = fl + List[row][col]/years
    Percentage = (Output*100)/(years*24*365)
    return(str(Percentage)+"%" + " Fails:" + str(temp[2])/100)

In [187]:
def create_tuple(List):
    Final_Fail = []
    data = openpyxl.load_workbook("Model_List_Updated_.xlsx")
    read_data = data.active
    i = -1
    for row in range(1, read_data.max_row):
        i +=1
        for col in read_data.iter_cols(1,1):
            temp_name = col[row].value
            Final_Fail.append([List[i],temp_name])
        if(i == 49):
            return Final_Fail

In [188]:
%%time

### THIS IS OLD CODE, IT STILL WORKS BUT NOT BEING USED FOR THE CURRENT SENSITIVIY ANALYSIS
#Fail_List_Binom = run_multi(years,4,sValue)
years = 100
#print(run_multi(years,1))
# Function to print a 3x3 table with string labels
data = [
    ["Exponential", run_multi(years,1), run_multi(years,0.75), run_multi(years,1.25)],
    ["Weibull", run_multi(years,2), run_multi(years,1.75), run_multi(years,2.25)],
    ["Binomial", run_multi(years,3), "N/A", "N/A"]
]

# Create a DataFrame with explicit column names
df = pd.DataFrame(data, columns=["Model", "Original", "-25%", "+25%"])

# Setting the 'Model' column as the index
df.set_index('Model', inplace=True)

# Output the DataFrame to an Excel file
excel_filename = "output.xlsx"
df.to_excel(excel_filename)

print(f"Table has been output to {excel_filename}")

# Prints the table output
#table = tabulate(data, headers="firstrow", tablefmt="grid")
#print(table)


#printTable(labels)

#print("Exponential:")
#Fail_List_Expon = run_multi(years,1)

#print("Exponential -25%:")
#Fail_List_ExponSA1 = run_multi(years,0.75)
#Fail_List_ExponSA2 = run_multi(years, 1.25)


#print("Weibull Only:")
#Fail_List_Weibull = run_multi(years,2)

#print("Weibull with 25% sensitivity analysis:")
#Fail_List_Weibull = run_multi(years,1.75)
#Fail_List_Weibull = run_multi(years, 2.25)


#Fail_Tup_Expon = create_tuple(Fail_List_Expon)
#Fail_Tup_Mix = create_tuple(Fail_List_Mix)
#Fail_Tup_Binom = create_tuple(Fail_List_Binom)

# Number of comps to be displayed
#print("Failed Items Expon")
#print("Amount     Name")
#i = 5
#while i > 0:
   # temp = max(Fail_Tup_Expon)
    #print(temp)
    #Fail_Tup_Expon.remove(temp)
   # i-=1

#i = 1
#print("Failed Items Weibull")
#print("Amount     Name")
#while i > 0:
    #temp = max(Fail_Tup_Weibull)
    #print(temp)
    #Fail_Tup_Weibull.remove(temp)
   # i-=1
    
#i = 1
#print("Failed Items Mix")
#print("Amount     Name")
#while i > 0:
  #  temp = max(Fail_Tup_Mix)
    #print(temp)
    #Fail_Tup_Mix.remove(temp)
    #i-=1

#i = 1
#print("Failed Items Binom")
#print("Amount     Name")
#while i > 0:
    #temp = max(Fail_Tup_Binom)
    #print(temp)
    #Fail_Tup_Mix.remove(temp)
   # i-=1

KeyboardInterrupt: 